<a href="https://colab.research.google.com/github/mamtakumari-hub/Predict-Candidate-Fit-Score/blob/main/Create_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

import sklearn
import joblib

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
candidates=pd.read_csv('/content/candidate_job_role_dataset.csv')
jobs=pd.read_csv('/content/naukri_data_science_jobs_india.csv')

candidates.head()
jobs.head()

,Job_Role,Company,Location,Job Experience,Skills/Description
0,Senior Data Scientist,UPL,"Bangalore/Bengaluru, Mumbai (All Areas)",3-6,"python, MLT, statistical modeling, machine lea..."
1,Senior Data Scientist,Walmart,Bangalore/Bengaluru,5-9,"Data Science, Machine learning, Python, Azure,..."
2,Applied Data Scientist / ML Senior Engineer (P...,SAP India Pvt.Ltd,Bangalore/Bengaluru,5-10,"Python, IT Skills, Testing, Cloud, Product Man..."
3,Data Scientist,UPL,"Bangalore/Bengaluru, Mumbai (All Areas)",1-4,"python, machine learning, Data Science, data a..."
4,Data Scientist,Walmart,Bangalore/Bengaluru,4-8,"IT Skills, Python, Data Science, Machine Learn..."


In [11]:
pip install fastapi uvicorn joblib pandas scikit-learn nltk


In [13]:
import joblib

model = joblib.load("job_candidate_matcher.pkl")


In [14]:
from fastapi import FastAPI
from pydantic import BaseModel
import joblib
import re
import nltk
from nltk.corpus import stopwords


nltk.download('stopwords')
STOP_WORDS = set(stopwords.words("english"))


def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'[^a-z ]', ' ', text)
    words = [w for w in text.split() if w not in STOP_WORDS]
    return " ".join(words)


model = joblib.load("job_candidate_matcher.pkl")

# FastAPI app

app = FastAPI(title="Job–Candidate Matching API")


class Candidate(BaseModel):
    skills: str
    qualification: str
    experience_level: str
    job_role: str

class Job(BaseModel):
    Job_Role: str
    Skills_Description: str


def explain_match(candidate, job, top_n=5):
    cand_words = set(clean_text(candidate.skills).split())
    job_words = set(clean_text(job.Skills_Description).split())
    return list(cand_words.intersection(job_words))[:top_n]


@app.post("/predict")
def predict(candidate: Candidate, job: Job):
    text = clean_text(
        candidate.skills + " " +
        candidate.qualification + " " +
        candidate.experience_level + " " +
        candidate.job_role + " " +
        job.Job_Role + " " +
        job.Skills_Description
    )
    prob = model.predict_proba([text])[0][1]
    keys = explain_match(candidate, job)
    return {"fit_score": round(prob*100, 2), "key_matching_factors": keys}


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
!uvicorn app:app --reload --host 0.0.0.0 --port 8000


INFO:     Will watch for changes in these directories: ['/content']
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [4496] using StatReload
ERROR:    Error loading ASGI app. Could not import module "app".
INFO:     Stopping reloader process [4496]


In [19]:
!ngrok authtoken 37RZRbuqgAMeWliemzF7KMf2Mab_uJ4xZQyRDeZV1ectJAaT


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [20]:
from pyngrok import ngrok
import uvicorn
import threading


public_url = ngrok.connect(8000)
print("Public URL:", public_url)

# Start FastAPI server in a thread
def run():
    uvicorn.run("app:app", host="0.0.0.0", port=8000)

threading.Thread(target=run).start()


Public URL: NgrokTunnel: "https://pseudoaccidentally-bigger-jackie.ngrok-free.dev" -> "http://localhost:8000"
